In [1]:
!pip install transformers seqeval[gpu]
!pip install simpletransformers
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from simpletransformers.ner import NERModel , NERArgs

In [6]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cuda


In [7]:
data = pd.read_csv('C:/Users/qwe11/PycharmProjects/Ner_task/ner_datasetreference.csv',encoding = 'unicode_escape')
data.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [8]:
data.count()

Sentence #      47959
Word          1048565
POS           1048575
Tag           1048575
dtype: int64

In [9]:
print('number of tags: {}'.format(len(data.Tag.unique())))
freq = data.Tag.value_counts()
freq

number of tags: 17


Tag
O        887908
B-geo     37644
B-tim     20333
B-org     20143
I-per     17251
B-per     16990
I-org     16784
B-gpe     15870
I-geo      7414
I-tim      6528
B-art       402
B-eve       308
I-art       297
I-eve       253
B-nat       201
I-gpe       198
I-nat        51
Name: count, dtype: int64

In [10]:
#ents_to_remove = ['B-art','B-eve','I-art','I-eve','B-nat','I-nat']
#data = data[~data.Tag.isin(ents_to_remove)] # Return DataFrame that doesn't contain the previous Features
#data.head()

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O


In [11]:
data = data.fillna(method = 'ffill')

In [13]:
data['Sentence #'] = LabelEncoder().fit_transform(data['Sentence #'])
data.head()

,Sentence #,Word,POS,Tag
0,0,Thousands,NNS,O
1,0,of,IN,O
2,0,demonstrators,NNS,O
3,0,have,VBP,O
4,0,marched,VBN,O


In [14]:
data.rename(columns = {'Sentence #':'sentence_id',
                       'Word':'words',
                       'Tag':'label'},inplace = True)

In [15]:
data

,sentence_id,words,POS,label
0,0,Thousands,NNS,O
1,0,of,IN,O
2,0,demonstrators,NNS,O
3,0,have,VBP,O
4,0,marched,VBN,O
...,...,...,...,...
1048570,42146,they,PRP,O
1048571,42146,responded,VBD,O
1048572,42146,to,TO,O
1048573,42146,the,DT,O


In [16]:
data['label'] = data['label'].str.upper()

In [17]:
X = data[['sentence_id','words']]
y = data['label']

In [18]:
X_train,X_test,y_train,y_test = train_test_split(X,y)

In [19]:
train_data = pd.DataFrame({"sentence_id":X_train["sentence_id"],"words":X_train["words"],"labels":y_train})
test_data = pd.DataFrame({"sentence_id":X_test["sentence_id"],"words":X_test["words"],"labels":y_test})

In [22]:
labels = data['label'].unique().tolist()
labels

['O',
 'B-GEO',
 'B-GPE',
 'B-PER',
 'I-GEO',
 'B-ORG',
 'I-ORG',
 'B-TIM',
 'I-PER',
 'I-GPE',
 'I-TIM']

In [23]:
#Определение гиперпараметров модели
args = NERArgs()
args.num_train_epochs = 1
args.learning_rate = 1e-4
args.overwrite_output_dir = True
args.train_batch_size = 32
args.eval_batch_size = 32

In [24]:
model = NERModel('bert','bert-base-cased',labels = labels , args = args)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['cl

In [25]:
model.train_model(train_data , eval_data = test_data , acc = accuracy_score)

  0%|          | 0/45 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/1498 [00:00<?, ?it/s]

C:\Users\qwe11\PycharmProjects\Ner_task\venv\lib\site-packages\torch\optim\lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "


(1498, 0.17820550251667586)

In [26]:
result, model_outputs, preds_list = model.eval_model(test_data)

  0%|          | 0/45 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1478 [00:00<?, ?it/s]

In [27]:
result

{'eval_loss': 0.15652676767711207,
 'precision': 0.835230276888827,
 'recall': 0.7732776833101458,
 'f1_score': 0.803060910513853}

In [32]:
prediction, model_output = model.predict(["Сторлица России -  Москва"])
prediction

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'Сторлица': 'B-ORG'}, {'России': 'O'}, {'-': 'O'}, {'Москва': 'I-PER'}]]